<a href="https://colab.research.google.com/github/QasimKhan5x/Evaluation-Exercise/blob/main/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai --upgrade

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from fastai import learner
from fastai.data import core
from fastai.metrics import mse
from fastai.callback import schedule

In [43]:
particles = pd.read_csv('monojet_Zp2000.0_DM_50.0_chan3.csv')
particles

,event ID,process ID,event weight,MET,METphi,obj1,E1,pt1,eta1,phi1,obj2,E2,pt2,eta2,phi2,obj3,E3,pt3,eta3,phi3,obj4,E4,pt4,eta4,phi4,obj5,E5,pt5,eta5,phi5,obj6,E6,pt6,eta6,phi6,obj7,E7,pt7,eta7,phi7,obj8,E8,pt8,eta8,phi8,obj9,E9,pt9,eta9,phi9
0,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;832841;-1.22431;b,947383.0,824498.0,-0.523103,1.64421;j,258722.0,243675.0,0.328962,2.25014;j,520092.0,108590.0,-2.24730,-1.85679;j,383024.0,88405.6,2.145160,-1.95635;j,39507.6,35365.1,0.470460,-1.16445;j,225430.0,26878.2,-2.81608,-2.25938;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;732115;1.17245;j,1070000.0,751597.0,0.858186,-1.84217;j,457647.0,110079.0,2.102870,2.17501;j,49649.1,30875.2,-1.03189,-1.7992;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;1.05628e+06;-3.06744;j,676000.0,640429.0,0.330450,0.704554;j,1240000.0,602104.0,1.343240,-0.550856;j,193865.0,44825.7,-2.14231,-0.965933;j,28217.2,24926.6,-0.454264,0.516676;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;631781;1.64181;j,936707.0,616229.0,0.973383,-1.56592;j,75998.0,52304.3,0.905489,-0.806736;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;457316;-1.87536;j,640313.0,589524.0,0.390749,1.23734;j,332790.0,126272.0,1.619980,-1.85524;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6394,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;914355;1.52903;j,1120000.0,1010000.0,0.453265,-1.77278;b,246762.0,95229.6,1.605110,-0.520258;j,402070.0,77864.5,2.32445,0.92948;j,74709.2,69979.3,0.359168,0.94458;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6395,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;782256;-1.20751;j,1920000.0,874869.0,1.418550,1.83927;j,156224.0,152161.0,-0.222571,-1.83395;j,102296.0,64861.3,1.02478,0.412515;j,421972.0,37149.5,-3.120860,2.50949;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6396,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;645619;-2.48399;j,787964.0,263158.0,1.759810,0.456786;j,408087.0,258903.0,-1.026900,1.53073;j,1430000.0,204716.0,-2.63018,-0.253528;j,75924.4,63520.6,-0.586197,0.84639;j,54710.9,47683.2,0.527799,-1.58165;j,173678.0,45568.1,-2.01196,1.10735;j,170938.0,28409.1,-2.47968,2.38334;j,43035.2,26973.3,-1.02803,0.105967;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6397,354;monojet_Zp2000.0_DM_50.0;2.44336e-05;632390;-0.975687;j,731849.0,722541.0,-0.134499,2.56393;j,268828.0,259144.0,0.153813,0.578428;j,153340.0,43931.5,1.92014,-0.237382;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Interpretation of the Dataset

From this dataset, I need to utilize 4D data given by E, pt, eta, and phi. Each event contains a variable number of objects. 

The original .csv file was not annotated with column names. So, I used Microsoft Excel to add column names locally up till the 4-momentum of obj9. 

Each event contains a variable number of objects. So, the initial objective is to extract all objects corresponding to jet particles i.e. the obj${_i}$ string ends with j. 

In [44]:
# meta columns that we don't need
meta = ['event ID',	'process ID', 'event weight', 'MET', 'METphi']
particles = particles.drop(meta, axis=1)
particles.shape

(6399, 45)

Initial inspection of the data shwos that it contains a lot of NaN values.

Let's take a closer look.

In [45]:
particles[particles['obj1'].isnull()]

,obj1,E1,pt1,eta1,phi1,obj2,E2,pt2,eta2,phi2,obj3,E3,pt3,eta3,phi3,obj4,E4,pt4,eta4,phi4,obj5,E5,pt5,eta5,phi5,obj6,E6,pt6,eta6,phi6,obj7,E7,pt7,eta7,phi7,obj8,E8,pt8,eta8,phi8,obj9,E9,pt9,eta9,phi9
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There is no data to be found whenver obj1 is NaN. So, we can drop all instances when obj1 is NaN.

In [47]:
particles = particles.dropna(subset=['obj1'])
particles = particles.reset_index(drop=True)
particles.head()

,obj1,E1,pt1,eta1,phi1,obj2,E2,pt2,eta2,phi2,obj3,E3,pt3,eta3,phi3,obj4,E4,pt4,eta4,phi4,obj5,E5,pt5,eta5,phi5,obj6,E6,pt6,eta6,phi6,obj7,E7,pt7,eta7,phi7,obj8,E8,pt8,eta8,phi8,obj9,E9,pt9,eta9,phi9
0,258722.0,243675.0,0.328962,2.25014;j,520092.0,108590.0,-2.24730,-1.85679;j,383024.0,88405.6,2.145160,-1.95635;j,39507.6,35365.1,0.47046,-1.16445;j,225430.0,26878.2,-2.81608,-2.25938;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,457647.0,110079.0,2.102870,2.17501;j,49649.1,30875.2,-1.03189,-1.7992;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1240000.0,602104.0,1.343240,-0.550856;j,193865.0,44825.7,-2.14231,-0.965933;j,28217.2,24926.6,-0.454264,0.516676;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,75998.0,52304.3,0.905489,-0.806736;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,332790.0,126272.0,1.619980,-1.85524;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# Check which objects correspond to jets
particles.apply(is_jet).sum().to_frame().transpose()

,obj1,E1,pt1,eta1,phi1,obj2,E2,pt2,eta2,phi2,obj3,E3,pt3,eta3,phi3,obj4,E4,pt4,eta4,phi4,obj5,E5,pt5,eta5,phi5,obj6,E6,pt6,eta6,phi6,obj7,E7,pt7,eta7,phi7,obj8,E8,pt8,eta8,phi8,obj9,E9,pt9,eta9,phi9
0,0,0,0,4771,0,0,0,3121,0,0,0,1641,0,0,0,671,0,0,0,248,0,0,0,73,0,0,0,19,0,0,0,5,0,0,0,1,0,0,0,1,0,0,0,0,0


From the above we see that a total of $671 + 1$ jets are found in obj4 and obj8.

Now, we need to separate each object into a separate instance.

The new dataframe will have 5 columns corresponding to obj, E, pt, eta, phi. Each instance of events will now occur 9 times (obj1 - obj9) for each object.

In [50]:
particles = particles.values.reshape([-1, 5])
particles = pd.DataFrame(particles, columns=['obj', 'E', 'pt', 'eta', 'phi'])
particles.head()

,obj,E,pt,eta,phi
0,258722,243675,0.328962,2.25014;j,520092
1,108590,-2.2473,-1.85679;j,383024,88405.6
2,2.14516,-1.95635;j,39507.6,35365.1,0.47046
3,-1.16445;j,225430,26878.2,-2.81608,-2.25938;
4,NaN,NaN,NaN,NaN,NaN


In [51]:
# Remove all NaN rows
particles.dropna(how='all', inplace=True)
particles

,obj,E,pt,eta,phi
0,258722,243675,0.328962,2.25014;j,520092
1,108590,-2.2473,-1.85679;j,383024,88405.6
2,2.14516,-1.95635;j,39507.6,35365.1,0.47046
3,-1.16445;j,225430,26878.2,-2.81608,-2.25938;
9,457647,110079,2.10287,2.17501;j,49649.1
...,...,...,...,...,...
55102,170938,28409.1,-2.47968,2.38334;j,43035.2
55103,26973.3,-1.02803,0.105967;,NaN,NaN
55107,268828,259144,0.153813,0.578428;j,153340
55108,43931.5,1.92014,-0.237382;,NaN,NaN


In [53]:
# Obtain all jet objects
jets = particles[particles['obj'].astype(str).str.split(';').apply(lambda x:
                                                               False if type(x) != type([])
                                                               else x[-1] == 'j')]
jets

,obj,E,pt,eta,phi
3,-1.16445;j,225430,26878.2,-2.81608,-2.25938;
48,-0.910782;j,28297.8,26177.9,-0.371397,-2.41833;
84,0.457269;j,60190.4,25060.9,-1.51968,-0.427073;
273,-2.01513;j,138699,26861.7,2.32418,-2.84795;
444,-0.643228;j,35547.3,33920.3,-0.118132,-0.853091;j
...,...,...,...,...,...
54858,-1.28031;j,39142,31483.8,0.679338,-3.09847;
54930,1.03644;j,90539.8,59787.3,0.961666,-1.23922;j
54966,3.06679;j,42767.5,36072,-0.571172,1.37815;
55029,-2.34551;j,102738,24401.5,2.11376,-2.30433;


In [55]:
jets.isna().sum()

obj    0
E      0
pt     0
eta    0
phi    0
dtype: int64

# Visualizations